In [1]:
import pandas as pd
import numpy as np
import ta
import talib
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import datetime

import util_functions as uf

In [2]:
sma_value = [50, 100, 150, 200]
close='Close'
sym='TSLA'

today = datetime.datetime.now()
end_date = datetime.datetime(today.year, today.month, today.day-1)
start_date = datetime.datetime(today.year-1, 1, 1)

In [3]:
df = uf.getdata(sym, sma_value, close, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [4]:
df['rsi'] = ta.momentum.rsi(df[close], window=50)
df.dropna(subset=['sma_50','rsi'],inplace=True)

In [5]:
def generate_signal(x):
    if (min(x['Open'], x['Close']) < x['sma_200']) & (x['rsi']<50):
        return 1
    elif (max(x['Open'], x['Close']) > x['sma_50']) & (x['rsi']>65): 
        return -1
    else:
        return 0
    

In [6]:
df['signal'] = df.apply(lambda row: generate_signal(row), axis=1)

In [7]:
df_trade = uf.get_df_trade_position(df, 'position','signal')

c:\Users\natal\OneDrive\Documents\Python Scripts\Quantitative_Projects\Stock_Analysis\Mean_Reverion_with_rsi\util_functions.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_position] = df[col_signal].diff()
c:\Users\natal\OneDrive\Documents\Python Scripts\Quantitative_Projects\Stock_Analysis\Mean_Reverion_with_rsi\util_functions.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_position] = df.apply(lambda row: update_position(row[col_position],row[col_signal]), axis=1)


In [8]:
df_trade

,Date,Open,High,Low,Close,Volume,ticker,sma_50,sma_100,sma_150,sma_200,rsi,signal,position
109,2023-06-09,249.070007,252.419998,242.020004,244.399994,199882300,TSLA,183.934199,184.2521,NaN,NaN,66.339796,-1,-1
201,2023-10-20,217.009995,218.860001,210.419998,211.990005,137734000,TSLA,249.113399,253.1465,228.484533,214.6719,44.747981,1,1


In [9]:
df = df.merge(df_trade[['Date','position']], how='left', on='Date')

In [10]:
df

,Date,Open,High,Low,Close,Volume,ticker,sma_50,sma_100,sma_150,sma_200,rsi,signal,position
0,2023-03-15,180.800003,182.339996,176.029999,180.449997,145995600,TSLA,170.383600,NaN,NaN,NaN,57.846910,0,NaN
1,2023-03-16,180.369995,185.809998,178.839996,184.130005,121136800,TSLA,171.904200,NaN,NaN,NaN,58.688065,0,NaN
2,2023-03-17,184.520004,186.220001,177.330002,180.130005,133197100,TSLA,173.234000,NaN,NaN,NaN,57.417273,0,NaN
3,2023-03-20,178.080002,186.440002,176.350006,183.250000,129684400,TSLA,174.692200,NaN,NaN,NaN,58.138721,0,NaN
4,2023-03-21,188.279999,198.000000,188.039993,197.580002,153391400,TSLA,176.382600,NaN,NaN,NaN,61.218126,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,2024-03-19,172.360001,172.820007,167.419998,171.320007,77271400,TSLA,194.606801,214.7672,225.857200,234.04260,41.651649,1,NaN
255,2024-03-20,173.000000,176.250000,170.820007,175.660004,83846700,TSLA,193.311001,214.3996,225.475200,233.85105,42.770529,1,NaN
256,2024-03-21,176.389999,178.179993,171.800003,172.820007,73178000,TSLA,192.068201,214.0702,225.123333,233.62710,42.229804,1,NaN
257,2024-03-22,166.690002,171.199997,166.300003,170.830002,75454700,TSLA,190.806001,213.7055,224.800733,233.37470,41.851489,1,NaN


In [11]:
def pointpos_pattern(x):
    if x['position']==-1:
        return x['High']+3
    elif x['position']==1:
        return x['Low']-3
    else:
        return np.nan
df['pointpos_pattern'] = df.apply(lambda row: pointpos_pattern(row), axis=1)

In [12]:
df.position.value_counts()

position
-1.0    1
 1.0    1
Name: count, dtype: int64

In [13]:
# Calculate Bollinger Bands
indicator_bb = ta.volatility.BollingerBands(close=df[close], window=20, window_dev=2)

# Get Bollinger Bands
df['bb_upper'] = indicator_bb.bollinger_hband()
df['bb_lower'] = indicator_bb.bollinger_lband()

In [14]:
df

,Date,Open,High,Low,Close,Volume,ticker,sma_50,sma_100,sma_150,sma_200,rsi,signal,position,pointpos_pattern,bb_upper,bb_lower
0,2023-03-15,180.800003,182.339996,176.029999,180.449997,145995600,TSLA,170.383600,NaN,NaN,NaN,57.846910,0,NaN,NaN,NaN,NaN
1,2023-03-16,180.369995,185.809998,178.839996,184.130005,121136800,TSLA,171.904200,NaN,NaN,NaN,58.688065,0,NaN,NaN,NaN,NaN
2,2023-03-17,184.520004,186.220001,177.330002,180.130005,133197100,TSLA,173.234000,NaN,NaN,NaN,57.417273,0,NaN,NaN,NaN,NaN
3,2023-03-20,178.080002,186.440002,176.350006,183.250000,129684400,TSLA,174.692200,NaN,NaN,NaN,58.138721,0,NaN,NaN,NaN,NaN
4,2023-03-21,188.279999,198.000000,188.039993,197.580002,153391400,TSLA,176.382600,NaN,NaN,NaN,61.218126,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,2024-03-19,172.360001,172.820007,167.419998,171.320007,77271400,TSLA,194.606801,214.7672,225.857200,234.04260,41.651649,1,NaN,NaN,210.559870,157.963129
255,2024-03-20,173.000000,176.250000,170.820007,175.660004,83846700,TSLA,193.311001,214.3996,225.475200,233.85105,42.770529,1,NaN,NaN,209.395534,157.216466
256,2024-03-21,176.389999,178.179993,171.800003,172.820007,73178000,TSLA,192.068201,214.0702,225.123333,233.62710,42.229804,1,NaN,NaN,207.705072,156.447927
257,2024-03-22,166.690002,171.199997,166.300003,170.830002,75454700,TSLA,190.806001,213.7055,224.800733,233.37470,41.851489,1,NaN,NaN,206.672476,155.366524


In [15]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.8, 0.2])

candlestick =   go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])

sma_50 = go.Scatter(x=df.index, y=df['sma_50'], mode='lines', name='sma_50', line=dict(color="MediumPurple"))
sma100 = go.Scatter(x=df.index, y=df['sma_100'], mode='lines', name='sma_100', line=dict(color="orange"))
sma150 = go.Scatter(x=df.index, y=df['sma_150'], mode='lines', name='sma_150', line=dict(color="pink"))
sma200 = go.Scatter(x=df.index, y=df['sma_200'], mode='lines', name='sma_200', line=dict(color="LightGreen"))
bb_upper = go.Scatter(x=df.index, y=df['bb_upper'], mode='lines', name='Upper Bollinger Band', line=dict(color="blue"))
bb_lower = go.Scatter(x=df.index, y=df['bb_lower'], mode='lines', name='Lower Bollinger Band', line=dict(color='blue'))

signal_sell =  go.Scatter(x=df[df['position']==1].index, y=df[df['position']==1]['pointpos_pattern'], mode='markers', name='signal', marker=dict(color="orange",symbol='arrow-bar-up'))
signal_buy =  go.Scatter(x=df[df['position']==-1].index, y=df[df['position']==-1]['pointpos_pattern'], mode='markers', name='signal', marker=dict(color="orange",symbol='arrow-bar-down'))

fig.add_trace(candlestick, row=1, col=1)
fig.add_trace(sma_50, row=1, col=1)
fig.add_trace(sma100, row=1, col=1)
fig.add_trace(sma150, row=1, col=1)
fig.add_trace(sma200, row=1, col=1)
fig.add_trace(bb_upper, row=1, col=1)
fig.add_trace(bb_lower, row=1, col=1)
fig.add_trace(signal_sell, row=1, col=1)
fig.add_trace(signal_buy, row=1, col=1)

rsi = go.Scatter(x=df.index, y=df['rsi'], mode='lines', name='rsi', line=dict(color='grey'))
rsi_overbought = go.Scatter(x=df.index, y=[60]*len(df), mode='lines', name='rsi_overbought', line=dict(color='blue'))
rsi_oversold = go.Scatter(x=df.index, y=[50]*len(df), mode='lines', name='rsi_oversold', line=dict(color='blue'))


fig.add_trace(rsi, row=2, col=1)
fig.add_trace(rsi_overbought, row=2, col=1)
fig.add_trace(rsi_oversold, row=2, col=1)

# fig.add_scatter(x=df.index, y=df['rsi'], mode='lines', name='rsi', line=dict(color="orange"))
# fig.add_scatter(x=df.index, y=df['pointpos'], mode="markers",
#                 marker=dict(size=5, color="MediumPurple", symbol='circle'),
#                 name="pivot")

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

In [16]:
fig =   go.Figure(data=[go.Scatter(x=df.index, y=df['rsi'], mode='lines', name='sma_50', line=dict(color="blue"))])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()